In [132]:
import numpy as np
import pickle
import scipy.stats as st
from scipy.optimize import minimize
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF, ConstantKernel, WhiteKernel

In [135]:
XX = np.load(r'D:\Network Security\KL Control\robotarium_python_simulator\rps\State_Data_Nominal.npy',allow_pickle=True)
UU = np.load(r'D:\Network Security\KL Control\robotarium_python_simulator\rps\Input_Dataset_new.npy',allow_pickle=True)

In [134]:
X = []
X_plot = []

X_Prev = []
X_plot_Prev = []

U = []
U_plot = []

for i in range(len(XX)):
    X.append(np.array(XX[i]))
    X_plot.append(np.array(XX[i]))

X = np.concatenate(X, axis=0)
X = np.reshape(X, (-1, 2))

U = []
for i in range(len(UU)):
    U.append(np.array(UU[i]))
    U_plot.append(np.array(UU[i]))

U = np.concatenate(U, axis=0)
U = np.reshape(U, (-1, 2))

In [131]:
X1 = XX[0,:,:,:]
X2 = XX[:2,:,:,:].reshape((-1,2))
X3 = XX[:,:,:,:].reshape((-1,2))

U1 = UU[0,:,:,:]
U2 = UU[:2,:,:,:].reshape((-1,2))
U3 = UU[:,:,:,:].reshape((-1,2))

In [169]:
# Dataset 1
X1 = np.hstack((XX[0,:-1])).reshape(-1,2)
U1 = np.hstack((UU[0,:])).reshape(-1,2)
Train1 = np.hstack((X1,U1))
Y1 = np.hstack((XX[0,1:])).reshape(-1,2)

# Dataset 2
X2 = np.hstack((XX[0,:-1],XX[1,:-1])).reshape(-1,2)
U2 = np.hstack((UU[0,:],UU[1,:])).reshape(-1,2)
Train2 = np.hstack((X2,U2))
Y2 = np.hstack((XX[0,1:],XX[1,1:])).reshape(-1,2)

# Dataset 3
X3 = np.hstack((XX[0,:-1],XX[1,:-1],XX[2,:-1])).reshape(-1,2)
U3 = np.hstack((UU[0,:],UU[1,:],UU[2,:])).reshape(-1,2)
Train3 = np.hstack((X3,U3))
Y3 = np.hstack((XX[0,1:],XX[1,1:],XX[2,1:])).reshape(-1,2)

In [182]:
def nominal_model_step(x, velocities, time_step):
    poses = np.zeros((2, 1))
    poses[0] = 1.0 * x[0] + time_step * velocities[0] + 0.0 * x[0]
    poses[1] = 1.0 * x[1] + time_step * velocities[1] + 0.0* x[1]
    return poses

In [231]:
# Define the GP model with RBF kernel
# Define the kernel for the Gaussian Process
kernel = ConstantKernel() * RBF(length_scale=[0.0,.0,.0,.0]) + WhiteKernel()
gp = GaussianProcessRegressor(kernel=kernel, n_restarts_optimizer=10)

# Train the GP model
gp.fit(Train2, Y2)

c:\Users\jamal\anaconda3\envs\mbrl\lib\site-packages\sklearn\gaussian_process\kernels.py:285: RuntimeWarning: divide by zero encountered in log
  return np.log(np.hstack(theta))
c:\Users\jamal\anaconda3\envs\mbrl\lib\site-packages\sklearn\gaussian_process\_gpr.py:629: ConvergenceWarning: lbfgs failed to converge (status=2):
ABNORMAL_TERMINATION_IN_LNSRCH.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  _check_optimize_result("lbfgs", opt_res)
c:\Users\jamal\anaconda3\envs\mbrl\lib\site-packages\sklearn\gaussian_process\kernels.py:420: ConvergenceWarning: The optimal value found for dimension 0 of parameter k2__noise_level is close to the specified lower bound 1e-05. Decreasing the bound and calling fit again may find a better value.
  warnings.warn(


GaussianProcessRegressor(kernel=1**2 * RBF(length_scale=[0, 0, 0, 0]) + WhiteKernel(noise_level=1),
                         n_restarts_optimizer=10)

In [ ]:
X_train = np.hstack((X2[:-1,:],U2[:,:])).reshape((-1,4))
Y_train = X2[1:,:].reshape((-1,2))

# Define the GP model with RBF kernel
# Define the kernel for the Gaussian Process
kernel = ConstantKernel() * RBF(length_scale=[0.5,0.5,0.5,0.5]) + WhiteKernel()
gp = GaussianProcessRegressor(kernel=kernel, n_restarts_optimizer=10)

# Train the GP model
gp.fit(X_train, Y_train)


In [208]:
gp3 = pickle.load(open("GP_nominal_2.dump","rb"))

In [230]:
# Example of predicting with the GP model
time_step = 0.0333
test_state = np.array([1.0, -.8])
test_velocities = np.array([-0.15, 0.15])
test_input = np.hstack((test_state, test_velocities)).reshape(1, -1)
predicted_next_state,covn = gp.predict(test_input,return_cov=True)

next_state_test = nominal_model_step(test_state, test_velocities, time_step)

print("Predicted next state:", predicted_next_state.T)
print('actual next state:',next_state_test)
print('Covariance:',covn)

Predicted next state: [[ 0.99915069]
 [-0.79607165]]
actual next state: [[ 0.995005]
 [-0.795005]]
Covariance: [[[1.24258659e-05 1.24258659e-05]]]


In [10]:
gp.kernel_.get_params()

{'k1': 9.8**2 * RBF(length_scale=[20.6, 19.4, 401, 417]),
 'k2': WhiteKernel(noise_level=1e-05),
 'k1__k1': 9.8**2,
 'k1__k2': RBF(length_scale=[20.6, 19.4, 401, 417]),
 'k1__k1__constant_value': 96.10841278607474,
 'k1__k1__constant_value_bounds': (1e-05, 100000.0),
 'k1__k2__length_scale': array([ 20.61778115,  19.42858169, 401.04667891, 416.52370577]),
 'k1__k2__length_scale_bounds': (1e-05, 100000.0),
 'k2__noise_level': 9.999999999999997e-06,
 'k2__noise_level_bounds': (1e-05, 100000.0)}

In [113]:
with open("GP_nominal_2.dump","wb") as f:
    pickle.dump(gp, f)

In [172]:
def calculate_kl_divergence(mu1, cov1, mu2, cov2):
    """
    Calculate the Kullback-Leibler (KL) divergence between two multivariate Gaussian distributions.

    Args:
        mu1: numpy array of shape (d,), the mean vector of the first Gaussian distribution
        cov1: numpy array of shape (d, d), the covariance matrix of the first Gaussian distribution
        mu2: numpy array of shape (d,), the mean vector of the second Gaussian distribution
        cov2: numpy array of shape (d, d), the covariance matrix of the second Gaussian distribution

    Returns:
        kl_divergence: float, the KL divergence between the two Gaussian distributions
    """
    d = len(mu1)

    # Invert the covariance matrix of the second Gaussian distribution
    cov2_inv = np.linalg.inv(cov2)

    # Calculate the trace term
    trace_term = np.trace(np.matmul(cov2_inv, cov1))

    # Calculate the squared difference in means
    mean_diff = mu2 - mu1
    mean_diff_term = np.dot(np.dot(mean_diff, cov2_inv), mean_diff)

    # Calculate the log-determinant term
    log_det_term = np.log(np.linalg.det(cov2) / np.linalg.det(cov1))

    # Calculate the KL divergence
    kl_divergence = 0.5 * (trace_term + mean_diff_term - d + log_det_term)

    return kl_divergence

In [229]:
Cov = np.array([[0.001, 0.0002], [0.0002, 0.001]])
Covn= np.diag([covn[0,0,0],covn[0,0,1]])
cov_reference = np.array([[0.0001, 0.00002], [0.00002, 0.0001]])

dkl = calculate_kl_divergence(next_state_test.reshape((2,)),Cov,predicted_next_state.reshape((2,)),Covn)
print(dkl)
np.clip(calculate_kl_divergence(np.array([-1.4,-0.8]).reshape((2,)),cov_reference,predicted_next_state.reshape((2,)),Covn),0.0,100)

75.84707870552552


100.0